In [1]:
from tqdm.auto import tqdm
import datasets

In [2]:
# Supress progress bars which appear every time a task is downloaded
datasets.utils.logging.set_verbosity_error()

In [4]:
dataset = datasets.load_dataset("lexglue", "unfair_tos")
dataset["train"].to_pandas()

,answer,index,text
0,Other,0,"last updated date : may 15 , 2017"
1,Arbitration,1,arbitration notice : unless you opt out of arb...
2,Contract by using,2,"you acknowledge and agree that , by accessing ..."
3,Unilateral change,3,"academia.edu reserves the right , at its sole ..."
4,Unilateral termination,4,academia.edu reserves the right to suspend or ...
5,Limitation of liability,5,neither academia.edu nor any other person or e...
6,Choice of law,6,these terms and any action related thereto wil...
7,Jurisdiction,7,the exclusive jurisdiction and venue of any ip...
8,Content removal,8,amazon reserves the right ( but not the obliga...


In [ ]:
import openai
import datasets
from tqdm.auto import tqdm
from legalbench.utils import generate_prompts
from legalbench.evaluation import evaluate

# --- 1. Configuración del Cliente OpenAI/Ollama y LLM ---
GENERATIVE_MODEL = "qwen3:8b" # O el modelo que estés usando en Ollama
TASK_NAME = "unfair_tos"

client = openai.OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama', # Ollama usa "ollama" o cualquier string si no se requiere auth
)

# --- 2. Cargar Datos de la Tarea y Plantilla de Prompt ---
# Cargar el conjunto de datos para la tarea específica desde Hugging Face
print(f"Cargando dataset para la tarea: {TASK_NAME}...")
dataset = datasets.load_dataset("lex_glue", TASK_NAME)
test_df = dataset["test"].to_pandas()
test_df = test_df[:100]
# train_df = dataset["train"].to_pandas() # Para few-shot si los quieres añadir al prompt base

# Cargar la plantilla de prompt base para la tarea
# Asegúrate de que la ruta 'tasks/TASK_NAME/base_prompt.txt' sea correcta
# relativa a donde ejecutas tu script/notebook.
prompt_template_path = f"legalbench/tasks/{TASK_NAME}/base_prompt.txt"
print(f"Cargando plantilla de prompt desde: {prompt_template_path}...")
try:
    with open(prompt_template_path) as f:
        prompt_template = f.read()
except FileNotFoundError:
    print(f"Error: No se encontró el archivo de prompt en {prompt_template_path}")
    print("Asegúrate de que la carpeta 'tasks' esté en el mismo directorio que tu script/notebook,")
    print("o ajusta la ruta según sea necesario.")
    exit()

# --- 3. Generar Prompts para los Datos de Prueba ---
# La función generate_prompts llenará las plantillas con los datos de test_df
print("Generando prompts para el conjunto de prueba...")
prompts_for_llm = generate_prompts(prompt_template=prompt_template, data_df=test_df)
print(f"Se generaron {len(prompts_for_llm)} prompts.")
#print(prompts_for_llm[0])
if not prompts_for_llm:
    print("No se generaron prompts. Revisa tu plantilla y datos.")
    exit()

# Mostrar un ejemplo del prompt generado (opcional)
# print("\nEjemplo de prompt generado:")
# print(prompts_for_llm[0])

# --- 4. Obtener Generaciones (Predicciones) del LLM ---
llm_generations = []
print(f"\nObteniendo predicciones del LLM ({GENERATIVE_MODEL}) para {len(prompts_for_llm)} instancias...")

# Es buena idea definir un system prompt general si tu modelo responde mejor con uno.
# Para LegalBench, los prompts suelen ser autocontenidos con instrucciones y ejemplos.
# El base_prompt.txt de abercrombie ya tiene ejemplos
# por lo que un system prompt vacío o muy genérico podría ser suficiente.
SYSTEM_PROMPT = """You are a clause classification assistant. "
    "Given a clause, classify it into one of the following categories: "
    "Arbitration, Unilateral change, Content removal, Jurisdiction, Choice of law, "
    "Limitation of liability, Unilateral termination, Contract by using, Other. "
    "Respond with ONLY the category name and nothing else."""
# O puedes dejarlo vacío si el prompt base es suficiente:
# SYSTEM_PROMPT = ""

#prompts_for_llm = prompts_for_llm[:15]

for user_prompt_content in tqdm(prompts_for_llm):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt_content}
    ]
    try:
        response = client.chat.completions.create(
            model=GENERATIVE_MODEL,
            messages=messages,
            temperature=0.0  # Generalmente bueno para tareas de clasificación/extracción
        )
        generated_text = response.choices[0].message.content.strip()
        # NUEVO: Añadir paso de post-procesamiento
        cleaned_generation = ""
        possible_labels = [
            "Arbitration", "Unilateral change", "Content removal", "Jurisdiction",
            "Choice of law", "Limitation of liability", "Unilateral termination",
            "Contract by using", "Other"
        ]

        # Intenta extraer la etiqueta si el LLM la incluye con "Label: "
        if "label:" in generated_text.lower():
            parts = generated_text.lower().split("label:")
            potential_label = parts[-1].strip()
            # Ahora verifica si esta etiqueta potencial es una de las conocidas
            # (esto ayuda si el LLM añade texto extra DESPUÉS de la etiqueta)
            for pl in possible_labels:
                if pl.lower() == potential_label: # Comparación exacta inicial
                    cleaned_generation = pl
                    break
                # Si no hay coincidencia exacta, intenta ver si la etiqueta es una subcadena
                # (por si el LLM dice "Label: Arbitration clause" en lugar de solo "Arbitration")
                if not cleaned_generation and pl.lower() in potential_label:
                     cleaned_generation = pl # Toma la primera que coincida como subcadena
                     # Podrías querer lógica más sofisticada aquí si hay ambigüedad

            # Si después de "Label:" no se encontró una etiqueta válida,
            # se podría intentar una búsqueda más general en todo el texto.
            if not cleaned_generation:
                # Fallback: buscar la última aparición de alguna etiqueta conocida en el texto
                # Esto es menos preciso y puede dar falsos positivos
                best_match = ""
                best_pos = -1
                for known_label in possible_labels:
                    pos = generated_text.lower().rfind(known_label.lower())
                    if pos > best_pos: # Encuentra la última aparición
                        best_pos = pos
                        best_match = known_label
                cleaned_generation = best_match

        else:
            # Si "Label:" no está, intenta encontrar la última etiqueta conocida mencionada
            best_match = ""
            best_pos = -1
            for known_label in possible_labels:
                # Buscamos la etiqueta completa para evitar coincidencias parciales no deseadas
                # (ej. "law" en "Choice of law" vs "contract by using law")
                # Usamos expresiones regulares para buscar palabras completas (case-insensitive)
                import re
                if re.search(r'\b' + re.escape(known_label.lower()) + r'\b', generated_text.lower()):
                    # Esta lógica es simple; si múltiples etiquetas están, puede no ser la correcta.
                    # La estrategia de abajo de buscar la última es una heurística.
                    pos = generated_text.lower().rfind(known_label.lower())
                    if pos > best_pos:
                        best_pos = pos
                        best_match = known_label
            cleaned_generation = best_match


        if not cleaned_generation: # Si aún no se pudo extraer
             print(f"WARN: No se pudo extraer una etiqueta válida de: '{generated_text[:100]}...' Se usará la salida original normalizada.")
             # En este caso, la normalización de evaluate() intentará limpiarla, pero probablemente falle.
             # O podrías asignar un placeholder como "extracción fallida"
             llm_generations.append(generated_text) # Usar el texto original para ver qué hace normalize()
        else:
             llm_generations.append(cleaned_generation)
    except Exception as e:
        print(f"Error al llamar al API de Ollama: {e}")
        # Decide cómo manejar errores: añadir un placeholder, reintentar, o parar.
        # Por ahora, añadiremos un string vacío para no romper la evaluación.
        llm_generations.append("") # O un valor que sepas que será incorrecto

if not llm_generations or len(llm_generations) != len(prompts_for_llm):
    print("Error: No se pudieron obtener todas las generaciones del LLM.")
    exit()


# --- 5. Evaluar las Generaciones ---
ground_truth_answers = test_df["answer"].tolist()

print("\nEvaluando las predicciones...")
# La función evaluate tomará el nombre de la tarea, las predicciones de tu LLM,
# y las respuestas correctas.
score = evaluate(TASK_NAME, llm_generations, ground_truth_answers)

print(f"\nResultado de la evaluación para la tarea '{TASK_NAME}' con el modelo '{GENERATIVE_MODEL}':")
print(f"Score: {score}")

Cargando dataset para la tarea: unfair_tos...
Cargando plantilla de prompt desde: legalbench/tasks/unfair_tos/base_prompt.txt...
Generando prompts para el conjunto de prueba...
Se generaron 100 prompts.

Obteniendo predicciones del LLM (qwen3:8b) para 100 instancias...


  0%|          | 0/100 [00:00<?, ?it/s]